In [1]:
#typical imports
import random
from tqdm import tqdm
import numpy as np
import os

#other imports
!pip install top2vec -q
from top2vec import Top2Vec
# import fasttext


     |████████████████████████████████| 15.3MB 265kB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 6.4MB 28.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2MB 40.0MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


FileNotFoundError: ignored

In [2]:
!pip uninstall numpy -q
!pip install numpy==1.19.5 -q

Proceed (y/n)? y
     |████████████████████████████████| 14.8MB 283kB/s 
ERROR: top2vec 1.0.24 has requirement numpy>=1.20.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
#train fastText

#cbow model :
ft_embeddings = fasttext.train_unsupervised('booking_sentences_processed.txt', model='cbow')

# Skipgram model :
# ft_embeddings = fasttext.train_unsupervised('fastText_training_data.txt', model='skipgram', verbose=True)


In [ ]:
ft_embeddings.save_model('fast_text_embeddings.bin')

In [3]:
#train doc2vec
!wget https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
documents = []
with open('booking_sentences_processed.txt', 'r') as f:
    documents = f.readlines()
# doc2vec = Top2Vec(documents, embedding_model='doc2vec', workers=8, document_ids=list(range(0,len(documents))), speed="deep-learn", keep_documents=False)
# doc2vec.save('doc2vec_top2vec.bin')

--2021-06-09 13:11:00--  https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8092842 (7.7M) [text/plain]
Saving to: ‘booking_sentences_processed.txt’

booking_sentences_p 100%[===================>]   7.72M  --.-KB/s    in 0.08s   

2021-06-09 13:11:00 (92.7 MB/s) - ‘booking_sentences_processed.txt’ saved [8092842/8092842]



In [ ]:
#Build model with USE
use = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
use.save('use_top2vec.bin')

In [ ]:
#Build model with SBert
from sentence_transformers import SentenceTransformer
#download and save the sbert model
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')
sbert.save("./stsb-mpnet-base-v2.pt")
#load and use SBert model
sbert = Top2Vec(documents, embedding_model_path='./stsb-mpnet-base-v2.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
sbert.save('sbert_top2vec.bin')

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Set up weights and biases to visualize the training of TSDAE
!pip install wandb -qqq
import wandb
wandb.login()
project='TSDAE-training-booking-reviews'

config={
    
}

In [5]:
#load sts dataset for evaluation
# import pandas as pd
# sts_df = pd.read_csv('sts-mt.csv')#, sep='\t')
# sts_df
with open('sts-mt.csv', 'r') as f:
  sts_lines = f.readlines() 

sts_df = []
for l in sts_lines:
  sts_df.append(l[:-1].split('\t'))

sentences1, sentences2, scores = [], [], []
for l in sts_df:
  scores.append(l[3])
  sentences1.append(l[4])
  sentences2.append(l[5])
#scores are from 0 to 5. So the following line normalizes the score
scores = [float(s)/5.0 for s in scores]

In [7]:
#evaluator for Sentennce Transformer
!pip install sentence_transformers -q
from sentence_transformers import evaluation
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 2.3MB 31.0MB/s 
     |████████████████████████████████| 1.2MB 35.6MB/s 
     |████████████████████████████████| 901kB 42.9MB/s 
     |████████████████████████████████| 3.3MB 41.9MB/s 


In [ ]:
#train TSDAE
!pip install sentence_transformers -q
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased' #can we use stsb-mpnet-base-v2?
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=False, pooling_mode_cls_token=True, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# model.to('cuda')

# Define a list with sentences (1k - 100k sentences)
train_sentences = documents

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
# train_dataset.to('cuda')

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

score_list = []
steps_list = []
#callback function
def call_back(score, epoch, steps):
  score_list.append(score)
  steps_list.append(steps)
  plt.plot(steps, score, alpha=0.7)
  plt.title("Training of TSDAE")
  plt.xlabel("Training Steps")
  plt.ylabel("Score")
  print("hehehe")

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True,
    save_best_model=True,
    checkpoint_path = './tsdae_checkpoints/',
    checkpoint_save_steps = 500, 
    checkpoint_save_total_limit = 2,
    evaluation_steps=1,
    callback=call_back,
    evaluator=evaluator

)
model.save('tsdae-model.pt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

In [7]:
score_list

[]

In [ ]:
#build top2vec model using tsdae
tsdae = Top2Vec(documents, embedding_model_path='./tsdae_model.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
tsdae.save('sbert_top2vec.bin')

In [ ]:
"""
@article{wang-2021-TSDAE,
    title = "TSDAE: Using Transformer-based Sequential Denoising Auto-Encoderfor Unsupervised Sentence Embedding Learning",
    author = "Wang, Kexin and Reimers, Nils and  Gurevych, Iryna", 
    journal= "arXiv preprint arXiv:2104.06979",
    month = "4",
    year = "2021",
    url = "https://arxiv.org/abs/2104.06979",
}
"""
